In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame').getOrCreate()

In [2]:
# create DataFrame through RDD

columns = ['currency', 'value']
inputdata = [('Euro', 90), ('Pound', 100), ('Yuan', 11), ('Yen', 2), ('US Dollar', 74), ('K Dinar', 242)]

# RDD
rdd = spark.sparkContext.parallelize(inputdata)
rddDF = rdd.toDF()
df = rddDF.withColumnRenamed('_1', 'Currency')
#df.show()
df = spark.createDataFrame(rdd).toDF(*columns)
#df.show()

In [3]:
# Create DataFrame

df = spark.createDataaFrame(data=inputdata, schema=columns)
#df.show()

In [4]:
#Writing df content to CSV

df.write.format('csv').save('D:/test1')

In [5]:
# write without partition
df.repartition(1).write.format('csv').save('D:/csv', header=True)

In [6]:
# Write DF content to text file through RDD
df.rdd.map(lambda x: x[0] + "," +str(x[1])).repartition(1).saveAsTextFile('D:/Text')

In [7]:
# Reading Data to DataFrame from datasource (csv)
df = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load('D:/csv/movie.csv')
#df.show()

AnalysisException: Path does not exist: file:/D:/Dataset/flight-data/csv/2015-summary.csv;

In [8]:
from pyspark.sql.functions import col, expr, column, udf, date_sub, date_add, col, datediff, regexp_extract
from pyspark.sql.types import StringType, IntegerType

# various ways of selecting columns

#df.select(col('DEST_COUNTRY_NAME')).show(5, False)

#df.select(column('DEST_COUNTRY_NAME')).show(5, False)

#df.select('DEST_COUNTRY_NAME').show(5, False)

#df.select('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME').show(5, False)

#df.select(expr('DEST_COUNTRY_NAME AS Destination')) .show(5, False)
#df.select('DEST_COUNTRY_NAME').show(5, False)


In [9]:
# add column to dataframe
df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME')).show(5, False)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|United States    |Romania            |15   |false        |
|United States    |Croatia            |1    |false        |
|United States    |Ireland            |344  |false        |
|Egypt            |United States      |15   |false        |
|United States    |India              |62   |false        |
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [10]:
# Adding column by user defined function

def computeGroup(count):
    if count < 2:
        return 'Min'
    elif count < 20:
        return 'Normal'
    elif count < 100:
        return 'More'
    else:
        return 'Busy'

group_udf = udf(computeGroup, StringType())
df.withColumn('Frequency', group_udf(`))).show(5, False)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Frequency|
+-----------------+-------------------+-----+---------+
|United States    |Romania            |15   |Normal   |
|United States    |Croatia            |1    |Min      |
|United States    |Ireland            |344  |Busy     |
|Egypt            |United States      |15   |Normal   |
|United States    |India              |62   |More     |
+-----------------+-------------------+-----+---------+
only showing top 5 rows



In [7]:
# Movie Dataset
from pyspark.sql.functions import col, expr, column, udf, date_sub, date_add, col, datediff, regexp_extract
from pyspark.sql.types import StringType, IntegerType
movieDf = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load('movie.csv')
movieDf.show()

movieYear = movieDf.withColumn('Year',regexp_extract(col('title'), r"(\d\d\d\d)", 1).cast(IntegerType()))
cleanedMovie = movieYear.na.drop()


+-------+--------------------+--------------------+
|movieId|               title|                type|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [8]:
def calDecade(years):
    return (years - years%10)

decadeudf = udf(calDecade, IntegerType())
movieDecade = cleanedMovie.withColumn('Decade', decadeudf(col('Year')).cast(IntegerType()))
movieDecade.show()

movieDecade.filter('Decade == 1990').show()

+-------+--------------------+--------------------+----+------+
|movieId|               title|                type|Year|Decade|
+-------+--------------------+--------------------+----+------+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|  1990|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|  1990|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|  1990|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|  1990|
|      5|Father of the Bri...|              Comedy|1995|  1990|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|  1990|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|  1990|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|  1990|
|      9| Sudden Death (1995)|              Action|1995|  1990|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|  1990|
|     11|American Presiden...|Comedy|Drama|Romance|1995|  1990|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|  1990|
|     13|        Balto (1995)|Adventure|